In [8]:
import numpy as np
import cv2

def scale_polygon(pts, scale):
    """
    Scale a polygon (Nx2 array of [x,y] points) about its centroid.
    :param pts: np.array of shape (N,2)
    :param scale: float, e.g. 1.2 for +20%, 0.8 for –20%
    :return: np.array of shape (N,2) of int coordinates
    """
    # compute centroid
    c = pts.mean(axis=0)
    # shift so centroid is origin, scale, shift back
    scaled = c + (pts - c) * scale
    return scaled.astype(np.int32)

# your original court
court = np.array([[508, 65], [796, 65], [1172, 567], [210, 569]])

# 20% smaller and larger buffers
buffer_small = scale_polygon(court, 0.9)
buffer_large = scale_polygon(court, 1.1)

# reshape for OpenCV
def to_cv_contour(pts):
    return pts.reshape(-1, 1, 2)

court_cnt   = to_cv_contour(court)
small_cnt   = to_cv_contour(buffer_small)
large_cnt   = to_cv_contour(buffer_large)
# draw them
img = cv2.imread('../CourtDetection/Images/img_0005.jpg')
mask_large = np.zeros(img.shape[:2],dtype=np.int32)
mask_small = np.zeros_like(mask_large)

cv2.fillPoly(mask_large, [large_cnt], 255)
cv2.fillPoly(mask_small, [small_cnt], 255)
ring_mask = cv2.bitwise_and(mask_large, cv2.bitwise_not(mask_small))


# 5. make a colored overlay of same shape as img
overlay = img.copy()
overlay[ring_mask == 255] = (0, 0, 100)   # red ring

# 6. alpha-blend back onto original
alpha = 0.3
img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

# original court in yellow
img = cv2.polylines(img, [court_cnt], True, (0,255,255), 2)
# smaller buffer in green
cv2.polylines(img, [small_cnt], True, (0,255,0), 2)
# larger buffer in red
cv2.polylines(img, [large_cnt], True, (0,255,0), 2)

cv2.imshow('buffers', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
